In [1]:
import torch
from torchvision import models


In [ ]:
torch_model = models.inception_v3(weights=True)
torch_model.eval()

In [3]:
# Load the image, resize it and convert it to a tensor and display it
from PIL import Image
from torchvision import transforms

img = Image.open('cat.jpg')
img = transforms.Resize((299, 299))(img)
img = transforms.ToTensor()(img)



In [4]:
y = torch_model(img.unsqueeze(0))

In [6]:
# Input to the model
x = img.unsqueeze(0)
# Export the model
torch.onnx.export(torch_model,               # model being run
                  # model input (or a tuple for multiple inputs)
                  x,
                  # where to save the model (can be a file or file-like object)
                  "out.onnx",
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names=['input'],   # the model's input names
                  output_names=['output'],  # the model's output names
                  dynamic_axes={'input': {0: 'batch_size'},    # variable length axes
                                'output': {0: 'batch_size'}})


In [7]:
import onnx

In [8]:
onnx_model = onnx.load("out.onnx")

In [9]:
onnx.checker.check_model(onnx_model)

In [10]:
import onnxruntime
import numpy as np

ort_session = onnxruntime.InferenceSession("out.onnx")


def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

In [12]:
# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}


In [16]:
import time
s = time.time()
ort_outs = ort_session.run(None, ort_inputs)
print(time.time() - s)

0.03857135772705078


In [18]:
# Run 100 times
s = time.time()
for _ in range(100):
    ort_outs = ort_session.run(None, ort_inputs)
print(time.time() - s)

3.9148426055908203


In [19]:
# compare ONNX Runtime and PyTorch results
np.testing.assert_allclose(
    to_numpy(y), ort_outs[0], rtol=1e-03, atol=1e-05)

print("Exported model has been tested with ONNXRuntime, and the result looks good!")


Exported model has been tested with ONNXRuntime, and the result looks good!
